In [8]:
# Imports for project
from io import BytesIO
import shutil
from ftplib import FTP
import os
import pandas as pd
import numpy as np
import math

In [10]:
# Controller for project

# get list of protein IDs to retrieve
with open('cullpdb_pc30_res3.0_R1.0_d191017_chains18877.gz') as fin: # get the list of ID's from the file
    IDs = []
    for line in fin:
        IDs += [line.split()[0].lower()]
IDs = IDs[1:] # remove file header

def getDF(idIn):
    import gzip### set up retreival of pdb file using input code
    ftp = FTP('ftp.wwpdb.org')
    ftp.login('username', 'password')
    flo = BytesIO()
    
    pdbID = idIn[0:4]
    chain = idIn[4]
    
    path = '/pub/pdb/data/structures/divided/pdb/'
    
    ftp.cwd(path + pdbID[1:3])
    #print('Retrieving pdb file for', pdbID)
    ftp.retrbinary('RETR pdb' + pdbID + '.ent.gz', flo.write)
    #print('File found')
    flo.seek(0)
    
    #print('Decompressing file')
    with open(pdbID + '.pdb', 'wb') as fout, gzip.GzipFile(fileobj = flo) as gzip:
        shutil.copyfileobj(gzip, fout)
    #print('Done')
    
    seq = ""
    helix = []
    sheet = []
    atom = []
    
    # extract data for the provided chain
    with open(pdbID + '.pdb', 'r') as fin:
        for line in fin:
            if line[0:6] == 'SEQRES' and line[11].lower() == chain:
                seq += line[19:]
            if line[0:6] == 'HELIX ' and line[19].lower() == chain:
                helix.append(line.split())
            if line[0:6] == 'SHEET ' and line[21].lower() == chain:
                sheet.append(line.split())
            if line[0:6] == 'ATOM  ' and line[21].lower() == chain:
                atom.append(line.split())
    seq = seq.split()
    
    
    
    #Remove the pdb file when finished
    #print('Removing pdb file for', pdbID)
    os.remove(pdbID+'.pdb')
    #print('Done')
    
    if seq == []:
        print('Chain', chain, 'does not exist.  Please try again with a different chain.')
        return []
    else:
        # store info in dataframes, and clean dataframes to contain only desired data
        #print('Creating DataFrames')
        if helix:
            helix = pd.DataFrame(helix)[[3,5,6,8,10]]
            helix.columns = ['initRes', 'initSeq', 'termRes', 'termSeq', 'length']
        else: helix = pd.DataFrame()
        sheet = pd.DataFrame(sheet)
        atom = pd.DataFrame(atom)[[2,3,5,6,7,8]]
        atom.columns = ['name', 'res', 'seqNum', 'x', 'y', 'z']
        #print('Done')
    return [helix, sheet, atom]

for ID in IDs:
    data = getDF(ID.lower())
    print(len(data[0]), len(data[1]), len(data[2]))

11 10 2559
11 3 2457
18 4 2206
4 0 498
8 15 2679
9 25 3202
1 9 881
4 4 879
1 0 256
7 2 1128
11 5 2073
16 5 3062
5 5 963
4 7 1245
13 8 2187
9 8 1811
2 0 423
46 37 8193
13 21 2889
6 8 1355
3 4 728
20 14 4546
4 7 908
7 8 1330
5 0 1161
10 13 1905
9 0 2032
1 2 925
1 9 947
10 0 1536
4 18 1686
13 12 2183
10 11 2038
6 0 1263
1 9 1374
6 6 1175
5 6 1377
1 12 1047
10 8 1785
11 32 4366
3 8 1057
2 7 1129
2 15 1339
10 10 1749
7 6 1382
8 9 1682
2 8 1127
3 0 475
4 0 1148
5 0 1003
2 11 1538
2 9 1018
9 8 1680
6 9 1484
2 0 263
30 15 4786
2 3 545
8 15 2029
8 12 2024
7 10 2242
2 0 419
43 0 4570
10 8 1973
16 20 3684
15 12 2618
5 15 1766
23 0 2524
0 4 726
17 10 2940
4 6 984
10 14 1952
3 7 701
2 1 401
1 0 258
4 5 685
10 13 2843
10 9 1769
5 13 1529
2 8 838
1 4 764
10 0 1168
5 9 1710
4 0 888
12 22 5735
21 13 4488
19 12 2592
8 0 1018
2 0 366
4 0 708
3 35 2813
8 15 2201
6 2 711
8 0 887
0 0 127
7 0 974
4 5 1007
4 2 1245
12 5 1907
5 12 2219
10 0 1029
4 14 1654
13 9 2146
17 11 2936
2 7 740
2 7 1318
2 6 864
17 19 351

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond